In [1]:
#from multiprocessing import Pool
#from functools import partial
import numpy as np
#from numba import jit

In [2]:
# load data
# from sklearn import datasets
# boston = datasets.load_boston()
# X = boston.data
# y = boston.target

# # train-test split
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [11]:
# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [12]:
'''
    pred() takes GBDT/RF outputs, i.e., the "score", as its inputs, and returns predictions.
    g() is the gradient/1st order derivative, which takes true values "true" and scores as input, and returns gradient.
    h() is the heassian/2nd order derivative, which takes true values "true" and scores as input, and returns hessian.
'''
class leastsquare(object):
    '''Loss class for mse. As for mse, pred function is pred=score.'''
    def pred(self,score):
        pred=score
        return pred

    def g(self,true,score):
        g = 2*(score-true)
        return g

    def h(self,true,score):
        h = np.ones(true.shape[0])
        h = 2*h
        return h

class logistic(object):
    '''Loss class for log loss. As for log loss, pred function is logistic transformation.'''
    
    def pred(self,score):
        pred = 1 / (1 + np.exp(-score))
        for i in range(len(pred)):
            if pred[i]>0.5:
                pred[i]=1
            else:
                pred[i]=0
        return pred

    def g(self,true,score):
        g = np.zeros(true.shape[0])
        #g = np.dot(X_train.T, (h - y))* (1/ y_train.shape[0])
        g = -true*np.exp(-score)/(1+np.exp(-score)) + np.exp(score)*(1-true)/(1+np.exp(score))
        return g

    def h(self,true,score):
        h = np.zeros(true.shape[0])
        h = true*np.exp(-score)/(1+np.exp(-score))**2 + np.exp(score)*(1-true)/(1+np.exp(score))**2
        return h
        

In [13]:
class RF(object):
    '''
    Class of Random Forest
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth d_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = None,
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0.2,
        rf = 0.99, num_trees = 100):
        
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.num_trees = num_trees
        self.trees = []
        self.idx = []

        for _ in range(self.num_trees):
            tree = Tree( n_threads=self.n_threads,max_depth = self.max_depth, min_sample_split=self.min_sample_split,
                        lamda=self.lamda,gamma=self.gamma,rf=1)
            self.trees.append(tree)
            
    def get_bootstrap_data(self, X, Y,g,h):

        n = X.shape[0]
        Y = Y.reshape(n, 1)
        tree_size = 100
        X_Y = np.hstack((X, Y))
        #np.random.shuffle(X_Y)
        data_sets = []
        for _ in range(self.num_trees):
            idm = np.random.choice(n, tree_size, replace=True)
            bootstrap_X_Y = X_Y[idm, :]
            bootstrap_X = bootstrap_X_Y[:, :-1]
            bootstrap_Y = bootstrap_X_Y[:, -1:]
            temp_g = g[idm]
            temp_h = h[idm]
            data_sets.append([bootstrap_X, bootstrap_Y,temp_g,temp_h])
        return data_sets
      
    def fit(self,train,target):
        y_hat0 = np.ones((len(y_train),1))*np.mean(target)# classify
        #y_hat0 = np.zeros((len(y_train),1)) # regression
        target = np.reshape(target,(len(target),1))
        loss = self.loss
        g = loss.g(target,y_hat0)
        h = loss.h(target,y_hat0)
        n_sample, m_feature = np.shape(train)
        sub_sets = self.get_bootstrap_data(train,target,g,h)
        for i in range(self.num_trees):
            sub_X, sub_Y,sub_g,sub_h = sub_sets[i]
            idx = np.random.choice(m_feature, 7, replace=False)
            self.idx.append(idx)
            sub_X = sub_X[:, idx]
            self.trees[i].fit(sub_X,sub_g,sub_h)
            #print("tree", i, "fit complete")
        return self
    
    def predict(self,test):
        
        scores = np.zeros((self.num_trees,len(test)))
        for i in range(self.num_trees):   
            scores[i,:] = self.trees[i].predict(test[:,self.idx[i]]).reshape(len(test))

        #scores = np.array(scores).reshape(self.num_trees,len(test))
        
        score = np.mean(scores,axis=0)
        return self.loss.pred(score)
    

In [14]:
class GBDT(object):
    '''
    Class of gradient boosting decision tree (GBDT)
   
    Parameters:
       n_threads: The number of threads used for fitting and predicting.
       loss: Loss function for gradient boosting.
           'mse' for regression task and 'log' for classfication task.
           A child class of the loss class could be passed to implement customized loss.
       max_depth: The maximum depth D_max of a tree.
       min_sample_split: The minimum number of samples required to further split a node.
       lamda: The regularization coefficient for leaf score, also known as lambda.
       gamma: The regularization coefficient for number of tree nodes, also know as gamma.
       learning_rate: The learning rate eta of GBDT.
       num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = None,
        max_depth = 5, min_sample_split = 10, 
        lamda = 0.1, gamma = 0.2,
        learning_rate = 0.005, num_trees = 100):
       
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.num_trees = num_trees
        self.tree = Tree(None,max_depth, min_sample_split,lamda, gamma, 0)
      
       
    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        y_prev = np.ones((len(y_train),1))*np.mean(target) #classify
        #y_prev = np.zeros((len(target),1)) #regression
        target = np.reshape(target,(len(target),1))
        loss = self.loss
        for i in range(self.num_trees):
            g = loss.g(target,y_prev)
            h = loss.h(target,y_prev)
            self.tree.fit(train,g,h)
            pred = self.tree.predict(train)
            y_prev += self.learning_rate * pred
            #print("tree",i,"fit complete")
        return self

    def predict(self, test):
        score=self.tree.predict(test)
        return self.loss.pred(score)

In [15]:
# TODO: class of a node on a tree
class TreeNode(object):
    '''
    Data structure that are used for storing a node on a tree.
    
    A tree is presented by a set of nested TreeNodes,
    with one TreeNode pointing two child TreeNodes,
    until a tree leaf is reached.
    
    A node on a tree can be either a leaf node or a non-leaf node.
    '''
    def __init__(self, split_feature=None, split_threshold=None, data=None,
                 left_child = None, right_child = None,weight=None):
        self.feature = split_feature  # Index for the feature that is tested
        self.threshold = split_threshold  # Threshold value for feature
        self.data = data
        self.left = left_child
        self.right = right_child
        self.weight = weight

In [16]:
class Tree(object):
    '''
    Class of a single decision tree in GBDT

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        max_depth: The maximum depth of the tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf prediction, also known as lambda.
        gamma: The regularization coefficient for number of TreeNode, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule,
            rf = 0 means we are training a GBDT.
    '''
    
    def __init__(self, n_threads = None, 
                 max_depth = 3, min_sample_split = 10,
                 lamda = 1, gamma = 0.2, rf = 0):
        self.n_threads = n_threads
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = 0
        self.root = TreeNode()
      
    def fit(self, train, g, h):
        '''
        train is the training data matrix, and must be numpy array (an n_train x m matrix).
        g and h are gradient and hessian respectively.
        '''
        self.root = self.construct_tree(train, g,h,self.max_depth,depth=0)
        
        return self
    
    def pred(self,node, row):  
        #print('node left:',node.left,'node right:',node.right,' r:',row[node.feature],'th:',node.threshold)
        #print(node.feature)
        if node.left==None and node.right==None:
            return node.weight
        if row[node.feature] < node.threshold:
            return self.pred(node.left, row)
        else:
            return self.pred(node.right, row)

    def predict(self,test):
        '''
        test is the test data matrix, and must be numpy arrays (an n_test x m matrix).
        Return predictions (scores) as an array.
        '''
        scores = np.zeros((len(test),1))
        
        for i in range(len(test)):
            #print('test',test[i,:])
            scores[i]=self.pred(self.root,test[i,:])
            #print('i',i,scores[i],'\n')
    
        return scores
     
    def construct_tree(self, train, g, h, max_depth, depth=0):
        '''
        Tree construction, which is recursively used to grow a tree.
        First we should check if we should stop further splitting.
        
        The stopping conditions include:
            1. tree reaches max_depth $d_{max}$
            2. The number of sample points at current node is less than min_sample_split, i.e., $n_{min}$
            3. gain <= 0
        '''
        n_samples = train.shape[0]
        
        #print(n_samples)
        
        feature, threshold, gain = self.find_best_decision_rule(train,g,h)
        
        if n_samples < self.min_sample_split or depth>max_depth or gain<=0:
            weight = -np.sum(g)/(np.sum(h)+self.lamda)
            #print('wwww',weight)
            return TreeNode(data = train,weight=weight)

        
        left = list()
        right = list()
        
        left_index = list()
        right_index = list()
        
        for i in range(n_samples):
          
            if train[i,feature]<threshold:
                left.append(train[i,:])
                left_index.append(i)
              
            else:
                right.append(train[i,:])
                right_index.append(i)
                
        
        left = np.array(left)
        right = np.array(right)

        if left.shape[0]==0 or right.shape[0]==0:
            weight = -np.sum(g)/(np.sum(h)+self.lamda)
            #print('wwww',weight)
            return TreeNode(data = train,weight=weight)

        left_child = self.construct_tree(left, g[left_index], h[left_index], max_depth, depth+1) 
        right_child = self.construct_tree(right, g[right_index], h[right_index],max_depth, depth+1)    
        #print('ft',feature,threshold)
        return TreeNode(split_feature = feature, split_threshold = threshold, data=train,
                    left_child = left_child, right_child = right_child)

    
    def find_best_decision_rule(self, train, g, h):
        '''
        Return the best decision rule [feature, treshold], i.e., $(p_j, \tau_j)$ on a node j, 
        train is the training data assigned to node j
        g and h are the corresponding 1st and 2nd derivatives for each data point in train
        g and h should be vectors of the same length as the number of data points in train
       
        for each feature, we find the best threshold by find_threshold(),
        a [threshold, best_gain] list is returned for each feature.
        Then we select the feature with the largest best_gain,
        and return the best decision rule [feature, treshold] together with its gain.
        '''
        threshold_all, best_gain_all = self.find_threshold(g, h, train)
        best_f_idx = np.argmax(best_gain_all)
        feature = best_f_idx 
        threshold = threshold_all[best_f_idx]
        gain = best_gain_all[best_f_idx]
        #print('ggggggggggggg',gain,best_gain_all)
        return feature, threshold, gain    
    
    def find_threshold(self, g, h, train):
        '''
        Given a particular feature $p_j$,
        return the best split threshold $\tau_j$ together with the gain that is achieved.
        '''
        threshold = []
        best_gain = []
        #np.shape(train)
        n_sample, m_feature = np.shape(train)
        
        for fe_i in range(m_feature):
            index_sort = np.argsort(train[:,fe_i])
            sort_g = g[index_sort]
            sort_h = h[index_sort]
            gain = []
            for i in range(n_sample):
                G_R = np.sum(sort_g[i:]) 
                G_L = np.sum(sort_g[:i])
                H_R = np.sum(sort_h[i:]) 
                H_L = np.sum(sort_h[:i])
                
                ga = 1/2* ((G_L**2/ (H_L + self.lamda))+(G_R**2/ (H_R + self.lamda)) - ((G_L+G_R)**2)/(H_L+H_R+self.lamda)) - self.gamma
                gain.append (ga)
                #print('GR,GL<HR<HL',G_R,G_L,H_R,H_L,ga)
            best_g_idx = np.argmax(gain)
#             print('bgi',best_g_idx)
            thold =  train[best_g_idx,fe_i]
            best_g = gain[best_g_idx]
            threshold.append(thold)
            best_gain.append(best_g)
            
        return [threshold, best_gain]

In [17]:
# RMSE
def root_mean_square_error(pred, y):
    rmse = np.sqrt(np.mean(np.square(y-pred)))
    return rmse

# precision
def accuracy(pred, y):
    error = 0
    for i in range(len(y)):
        if pred[i] != y[i]:
            error = error+1
    return 1 - error/len(y)

RF for classification on cancer dataset:

In [19]:
rf = RF(num_trees=100,loss=logistic())
rf.fit(X_train,y_train)
pred = rf.predict(X_train)
print('Accuracy for training data:',accuracy(pred,y_train))
pred = rf.predict(X_test)
print('Accuracy for test data:',accuracy(pred,y_test))

Accuracy for training data: 0.9396984924623115
Accuracy for test data: 0.9473684210526316


GBDT for classification on cancer dataset:

In [20]:
gbdt = GBDT(num_trees=100,loss=logistic())
gbdt.fit(X_train,y_train)
pred = gbdt.predict(X_train)
print('accuracy for training dataset:',accuracy(pred,y_train))
pred = gbdt.predict(X_test)
print('accuracy for test dataset:',accuracy(pred,y_test))

accuracy for training dataset: 0.9396984924623115
accuracy for test dataset: 0.9064327485380117
